# Unsupervised Learning

## 0 Recap on Vector Space and Metric 

**Vector space** ?



**Metric** ?

## 1 Clustering 

**Problem**:  

Given finite dataset $\{x_i\}_{i=1}^N$ find mapping $h$: $X \rightarrow \{1..K\}$,  
where $K$ is the number of clusters.

### 1.1 Clustering Metrics

Let $c_k$ be the cluster centers.

**Intracluster distance (unsupervised)**

$$D_{in} = \sum_{i=1}^N \sum_{k=1}^K [h(x_i) = k] d(x_i, c_k) $$

We want to minimize intracluster distance.

**Intercluster distance (unsupervised)**

$$D_{out} = \sum_{i=1}^N \sum_{j=1}^N [h(x_i) \neq h(x_j)] d(x_i, x_j) $$

We want to maximize intercluster distance.

**Sihouette (supervised)**

If you have some ground truth cluster assignments.  

Let $a$ - mean distance between the sample and other objects in the same class
$$a = \frac 1 {|S_{t(x)}|} \sum_{x_j \in t(x)} d(x, x_j)$$

Let $b$ - mean distance between the sample and the objects from the nearest cluster
$$b = \min_{k} \frac 1 {|S_k|} \sum_{x_j \in S_k, x \notin S_k} d(x, x_j)$$

$$S = \frac 1 N \sum_{i=1}^N s(x_i) = \frac 1 N \sum_{i=1}^N \frac {b(x_i) - a(x_i)} {\max(b(x_i), a(x_i))} $$

**F1-score (supervised)**

### 1.2 K-means

**Objective**


$$\sum_{i=1}^N \sum_{k=1}^K [x_i \in S_k] d(x_i, c_k) \rightarrow \min_{S_k}$$

We want assign clusters to minimize intercluster distance.

**Algorithm sketch**

1. Select initial cluster centers $c_k^{(0)}$  
1. Assign objects to clusters $$h(x) = \arg \min_{k} d(x, c_k)$$  
1. Update cluster centers $$c_k^{(t+1)} = \frac 1 {|S_k|} \sum_{x_i \in S_k} x_i $$ 
1. Repeat steps 2-3 until convergence

**Questions**:

* On which assumptions about data does K-means rely upon?  
* What is algorithm complexity?  
* Any ideas on good initialization?  
* How to select number of clusters k?

**Other versions**:

* K-means++ - select cluster centers iteratively, with probability to choose an object as a new cluster center  $$p(x_j = c_{k+1}) \sim \min_{k} d^2(x_, c_k)$$ 
* K-medoids - for non-vector space. For $c_k$ is always selected some object $x_i$ from the dataset. 

### 1.3 DBSCAN

density-based clustering with noise. 

**Algorithm sketch**:  

1. For each sample $x_i$ find neighbors in the sphere of radius $\epsilon$ so 
$$N(x) = \{x_j | d(x, x_i) \leq \epsilon \}$$  
1. Define core points as samples with more then a *min_pts* of neightbors within $\epsilon$-neighborhood. 
1. Find connected components of core points on the neighborhood graph, set them as clusters.  
1. Assign all non-core points to the nearest cluster if it is within $\epsilon$-neighborhood
1. All non-clustered points are noise.


Questions:  

1. Pros and cons of DBSCAN vs Kmeans? 

### 1.4 Hierarchical Clustering 

Clustering is perfomed from bottom to top.  



**Algorithm sketch**:  
1. Assign each object a its own cluster. 
1. Merge 2 closest clusters according to some linkage criteria until there is only 1 cluster left.  
1. Look at dendrogramm and select appropriate clustering.  


**Linkage criteria**:

distance between clusters A and B.

* Complete Linkage

$$\max_{a \in A, b \in B} d(a,b)$$

* Single Linkage

$$\min_{a \in A, b \in B} d(a,b)$$

* Average linkage

$$\frac 1 {|A||B|} \sum_{a \in A, b \in B} d(a,b)$$

* Ward linkage

$$ D_{in}(A + B) - D_{in}(A) - D_{in}(B) $$

## 2 Dimension Reduction

Given a set of points $X \in R^{NxD}$, find a map into a lower dimension subspace $f: R^D \rightarrow R^d$ so $d  << D$

Depending of the map function there are:  

1. Linear methods: PCA, SVD, NMF, etc..
1. Nonlinear methods

Sometimes, feature selection is considered as dimension reduction.

### 2.1 Random Projections

<img src="images/proj.png" style="height:300px">

$$ f(X) = X S $$
where $ S \in R^{Dxd}$

A common example is a gaussian random projection:
1. $S$ is sampled from a normal gaussian distribution
1. each row has unit norm $ ||S_{i,:} || = 1$
1. rows are pairwise orthogonal $ S_{i,:}^T S_{j,:} = 0 $ iff $i \neq j$



**Johnson - Lindenstrauss Lemma**:  

States, that a set of points in a high-dimensional space can be embedded into a space of much lower dimension in such a way that distances between the points are nearly preserved.  

Given $0 < \epsilon  < 1$, a set of points $X \in R^D$, number $d > \frac {8 \log(m)} {\epsilon^2}$, there exists a linear map $f: R^D \rightarrow R^d$ such that:  
$$ (1-\epsilon) ||u - v||^2 \leq ||f(u) - f(v)||^2 \leq (1+\epsilon) ||u - v||^2$$

There exists a set of points of size $m$ that needs dimension $O ( \frac {\log(m)} {\epsilon^2} )$ in order to preserve the distances between all pair of points.

### 2.2 PCA
Principal Component Analysis  

Preserve more information while reducing feature dimensions. Here we make assumption that variance is our measure of information quantity. More variable the feature is, more information it contains.

**Algorithm sketch (naive)**:

1. center data $\bar X = X - mean(X)$  
1. build covariance matrix $S = \bar X \bar X^T$  
1. make low-rank approximation for covariance matrix $S \approx \hat S_k = U_k \Sigma_k U_k^T$ by eigendecomposition
1. sort eigenvalues in descending order by their absolute value
1. select top-k eigenvalues and project data onto corresponding eigenvectors

**Learned mapping**:
$$f(X) = X U_k$$

**Algorithm sketch (practical)**:

1. center data $\bar X = X - mean(X)$  
1. find 1 principal component
$$\max_{w_1} ||Xw_1||^2$$
$$ ||w_1|| = 1 $$

2. find k-th principal component
$$\max_{w_k} ||Xw_k||^2$$
$$ ||w_k|| = 1 $$
$$ \forall i \in 1.. (k-1) ~~~ w_k^T w_i = 0  $$




**Properties**:
1. PCA basis vectors creates an uncorrelated orthogonal basis set.  
1. PCA is sensitive to feature scaling 

<img src="images/pca.png" style="height:300px">

### 2.3 Truncated SVD

**Learning objective**:  
$$ || X - \hat X_k||_F \rightarrow min$$
subject to $rank(X_k) = k$.  

which has the exact solution given by SVD decomposition
$$ X \approx \hat X_k = U_k \Sigma_k V^T$$

**Learned mapping**:   

$ \hat X = X V_k$

**Properties**:
1. Do not use covariance matrix
1. Do not center data
1. Preferable over PCA for sparse features

<img src="images/svd.png" style="height:300px">

### 2.4 NMF

Non-negative matrix Factorization

**Learnable objective**:

$$ || X - WH ||_F \rightarrow \min_{W, H}$$
subject to $W \geq 0$ and $H \geq 0$.

$$ X \approx \hat X_k = W H $$

Regularized with ElasticNet:  
$$ || X - WH ||_F + \gamma *( \alpha(||W||_1 + ||H||_1)  + \frac {1 - \alpha} 2 ( ||W||_2  + ||H||_2)) \rightarrow \min_{W, H} $$

## 2.5 Manifold Learning

Data lies on a low-dimensional non-linear manifold in a high-dimensional space.  
In other words, features are connected by some non-linear function.


<img src="images/manifold1.png" style="height:300px">

<img src="images/manifold2.png" style="height:300px">

### 2.6 AutoEncoders
Autoencoders are feed-forward neural networks, which contains encoder and decoder part. All training methods applicable to feed-forward NN, also can be applied to autoencoders. But unlike feed-forward NN, autoencoders are unsipervised models.

Let $E$ be encoder,  
$D$ - decoder,  
$L$  - some loss function.

Than, in the most simple case, the goal is to learn an efficient feature represetation by directing information through a bottleneck and predicting sample itself: 
$$ L(X, D(E(X))) \rightarrow \min_{D, E}$$


**Denoising Autoencoders**

$$ \bar X = X + \epsilon $$,
usually we use white noise $\epsilon \sim N(0,\sigma)$

$$ L(X, D(E(\bar X))) \rightarrow \min_{D, E}$$

<img src="images/autoencoder.png" style="height:300px">

## 3 Neighborhood Embedding

## 3.1 t-SNE

t-distributed stochastic neighbor embedding

**Algorithm sketch**:  
1. Compute probability that a sample $x_i$ would peak $x_j$ as its neighbour  
$$ p(j | i) = \frac {1} {Z} \exp (- \frac {|| x_i - x_j||_2^2} { 2 \sigma_i^2}) $$  
$Z = \sum_{k \neq i} \exp (- \frac {|| x_i - x_k||_2^2} { 2 \sigma_i^2}) $ - normalization factor

2. probability that $x_i$ and $x_j$ are neighbors:  
$$ p(i,j) = \frac {p(i | j)  + p(j | i)} {2N}$$  
but $p(i,i) = 0$

3. introduce map $Y \in R^d$, for wich probability that $y_i$ and $y_j$ are neighbors:   
$$ q(i, j) = \frac 1 Z (1 + || y_i - y_j||^2 )^{-1}$$
$Z = \sum_{k \neq i} (1 + || y_i - y_k||^2 )^{-1} $ - normalization factor

4. **Learning objective:**  
minimize Kullback–Leibler divergence between distributions:
$$ KL(P || Q) = \sum_{i \neq j} p(i,j) \log \frac {p(i,j)} {q(i,j)} \rightarrow \min_{Y} $$


Questions:
* Why is this paricular form of $q(i,j)$?